In this notebook, the performance of 3 deep nn on time series data of stock markets are compared:
* RNN
* LSTM 
* GRU

In [ ]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from torch.autograd import Variable
from io import open
import csv
from math import ceil, floor

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [4]:
fileName = 'sample_data/data.csv'
data = pd.read_csv(fileName)
data = data.iloc[:,0:1].values
sc = MinMaxScaler()
data = sc.fit_transform(data)
numSamples = len(data)

In [5]:
# Class: RNN
class RNN(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim,n_layers):
    super(RNN,self).__init__()
    self.hidden_dim = hidden_dim
    self.rnn = nn.RNN(input_dim, hidden_dim, n_layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self,x,hidden):
     batch_size = x.size(0)
     # RNN outputs
     r_out, hidden = self.rnn(x, hidden)
     # shape output to be (batch_size*seq_length, hidden_dim)
     r_out = r_out.view(-1, self.hidden_dim)  
     output = self.fc(r_out)
     return output, hidden

In [32]:
#Class: LSTM
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.1):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.fc(self.relu(out[:,-1]))
        return out, hidden

In [7]:
#Class: GRU
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, hidden):
        out, hidden = self.gru(x, hidden)
        out = self.fc(self.relu(out[:,-1]))
        return out, hidden
    

In [8]:
criterion = nn.MSELoss()
seq_length = 20
input_dim=1 
output_dim=1
hidden_dim=32
num_layers=1

n_steps = floor(numSamples/seq_length)
train_steps = int(n_steps * 0.7)
test_steps = n_steps - train_steps
print_every = 15

In [9]:
def train(data, model_type="RNN"):
  hidden = None
  if model_type == "RNN":
      model = RNN(input_dim, hidden_dim, output_dim, num_layers)
  elif model_type == "LSTM":
      model = LSTM(input_dim, hidden_dim, output_dim, num_layers)
  else:
      model = GRU(input_dim, hidden_dim, output_dim, num_layers)
  
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  model.to(device)  
  model.train()
  print("Starting Training of {} model".format(model_type))    
  for batch_i,step in enumerate(range(train_steps)):
    firtInd = step*seq_length
    lastInd = (step+1)*seq_length-1

    x = data[firtInd:lastInd-1]
    y = data[firtInd+1:lastInd]
    # convert data into Tensors
    x_tensor = torch.Tensor(x).unsqueeze(0) # unsqueeze gives a 1, batch_size dimension
    y_tensor = torch.Tensor(y)

    prediction, hidden = model(x_tensor, hidden)

    if model_type == "RNN" or model_type == "GRU":
      hidden = hidden.data
    else:
        hidden = tuple([e.data for e in hidden])

    loss = criterion(prediction, y_tensor)
    # zero gradients
    optimizer.zero_grad()
    # perform backprop and update weights
    loss.backward()
    optimizer.step()

    print('Loss: ', loss.item())
    # if batch_i%print_every == 0:        
    #   print('Loss: ', loss.item())
  return model

In [13]:
def eval(model):
  loss_val = 0
  model.eval()
  hidden = None
  for i,step in enumerate(range(test_steps)):
    firtInd = step*seq_length
    lastInd = (step+1)*seq_length-1

    if step == test_steps:
      lastInd = numSamples-1
    x = data[firtInd:lastInd-1]
    y = data[firtInd+1:lastInd]
    # convert data into Tensors
    x_tensor = torch.Tensor(x).unsqueeze(0) # unsqueeze gives a 1, batch_size dimension
    y_tensor = torch.Tensor(y)

    predict,h = model(x_tensor,hidden)
    pred = torch.Tensor(predict.data[0])
    # print(predict)
    # print(y_tensor.size())
    loss = criterion(pred, y_tensor)
    loss_val = loss_val + loss
  return loss_val/test_steps*100

In [20]:
modelRNN = train(data, model_type="RNN")

Starting Training of RNN model
Loss:  0.008223269134759903
Loss:  0.03255753591656685
Loss:  0.004472177010029554
Loss:  0.0017064898274838924
Loss:  0.013760387897491455
Loss:  0.025975549593567848
Loss:  0.021533692255616188
Loss:  0.012615122832357883
Loss:  0.0008284252835437655
Loss:  0.004725311882793903
Loss:  0.02348748967051506
Loss:  0.005808983463793993
Loss:  0.12506693601608276
Loss:  0.022986633703112602
Loss:  0.00161171134095639
Loss:  0.044263340532779694
Loss:  0.07729709893465042
Loss:  0.058353081345558167
Loss:  0.02260841801762581
Loss:  0.0003921735333278775
Loss:  0.0017209808574989438
Loss:  0.015011615119874477
Loss:  0.05842477083206177
Loss:  0.10018345713615417
Loss:  0.09193155169487


In [ ]:
modelLSTM = train(data, model_type="LSTM")

In [22]:
modelGRU = train(data, model_type="GRU")

Starting Training of GRU model
Loss:  0.010608199052512646
Loss:  0.00012908682401757687
Loss:  0.0034982110373675823
Loss:  0.003956187050789595
Loss:  0.004441886208951473
Loss:  0.00782769825309515
Loss:  0.007343041244894266
Loss:  0.008513342589139938
Loss:  0.0012439859565347433
Loss:  0.018518198281526566
Loss:  0.07626750320196152
Loss:  0.11205262690782547
Loss:  0.011209335178136826
Loss:  0.0034012855030596256
Loss:  0.03580725938081741
Loss:  0.07558821141719818
Loss:  0.37011078000068665
Loss:  0.30629438161849976
Loss:  0.20346802473068237
Loss:  0.0927707627415657
Loss:  0.056376561522483826
Loss:  0.02185996063053608
Loss:  0.0008381365332752466
Loss:  0.019788645207881927
Loss:  0.038711067289114


In [34]:
errRNN = eval(modelRNN)
errLSTM = eval(modelLSTM)
errGRU = eval(modelGRU)
print('The final error of RNN is: {:.4f}'.format(errRNN))
print('The final error of LSTM is: {:.4f}'.format(errLSTM))
print('The final error of GRU is: {:.4f}'.format(errGRU))

The final error of RNN is: 3.5953
The final error of LSTM is: 7.5821
The final error of GRU is: 2.8937
